In [1]:
import openai
import csv
import os
import openai
import pandas as pd
import collections
import ast
import time
import collections
import json
import sqlite3
import tiktoken
import xxhash
import numpy as np
from openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
import pickle 

In [2]:
os.environ["OPENAI_API_KEY"] = ""

openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
def save(fname, d):
    with open(fname, 'wb') as f:
        pickle.dump(d, f)
def clean_query(sql_query):
    sql_query = sql_query.replace("```sql", '')
    sql_query = sql_query.replace("`", '')
    sql_query = sql_query.replace(';', '')
    sql_query = sql_query.replace('"""', '')
    return sql_query
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [4]:
import time as time
from multiprocessing import Process, Queue
import query_module

def evalfunc(sql_source, sql_target, database, source='kaggle'):
    assert source in ['kaggle', 'bird']
    db_path = ''
    if source == 'kaggle':
        db_path = f'./databases/{database}/{database}.sqlite'
    elif source == 'bird':
        db_path = f'./bird_dev/dev_databases/{database}/{database}.sqlite'
    if not os.path.isfile(db_path):
        print("cannot find file", db_path)
        return False
    timeout = 120
    output = Queue()
    query_process = Process(target=query_module.execute_query, args=(db_path, sql_source, output))
    query_process.start()
    output_hash = ''
    
    try:
        # Connect to sqlite db
        # Execute both!
        source_results = None
        source_results = output.get(True, timeout+5)
        query_process.join(timeout)
        if query_process.is_alive():
            print("process terminated")
            query_process.terminate()  # Terminate the process
            query_process.join()  # Make sure it's cleaned up
            return False, [Exception('SQL query took too much time to execute.')]
        if isinstance(source_results, Exception):
            raise source_results
        output_hash = xxhash.xxh128_hexdigest(str(len(source_results)), seed=123)
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        target_results = cursor.execute(sql_target).fetchall()
        cursor.close()
        connection.close()
        # If the lengths don't match... there's no hope
        if len(source_results) != len(target_results):
            # (result matches or not, valid, hash)
            return False, []
        if 'ORDER BY' in sql_target:
            for a, b in zip(source_results, target_results):
                # NOTE: we are doing compares that are column-order independent
                # hence the sorting and the weird key (since we may have mixed
                # types in a row)
                lhs = tuple(sorted(list(a), key=lambda x: hash(x)))
                rhs = tuple(sorted(list(b), key=lambda x: hash(x)))
                output_hash = xxhash.xxh128_hexdigest(output_hash + str(lhs), seed=123)
                if lhs != rhs:
                    # Oh no, a row doesn't match!
                    return False, []
        else:
            lset, rset = set(), set()
            for a, b in zip(source_results, target_results):
                # NOTE: we are doing compares that are column-order independent
                # hence the sorting and the weird key (since we may have mixed
                # types in a row)
                lset.add(tuple(sorted(list(a), key=lambda x: hash(x))))
                rset.add(tuple(sorted(list(b), key=lambda x: hash(x))))
            output_hash = xxhash.xxh128_hexdigest(str(lset), seed=123)
            if lset != rset:
                # Oh no, rows don't match!
                return False, []
    # If we hit an error, that's not a match I guess...
    except Exception as ex:
        print(ex)
        return False, [ex]
    return True, []


def outputHash(sql_source, database):
    db_path = f'./databases/{database}/{database}.sqlite'
    output_hash = ''
    try:
        # Connect to sqlite db
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        source_results = cursor.execute(sql_source).fetchall()
        output_hash = xxhash.xxh128_hexdigest(str(len(source_results)), seed=123)
        if 'ORDER BY' in sql_source:
            for a in source_results:
                lhs = tuple(sorted(list(a), key=lambda x: hash(x)))
                output_hash = xxhash.xxh128_hexdigest(output_hash + str(lhs), seed=123)
        else:
            lset = set()
            for a in source_results:
                lset.add(tuple(sorted(list(a), key=lambda x: hash(x))))
            output_hash = xxhash.xxh128_hexdigest(str(lset), seed=123)
    except Exception as ex:
        return False
    finally:
        cursor.close()
        connection.close()
    return output_hash


def execute(sql, database, source):
    assert source in ['kaggle', 'bird']
    db_path = ''
    if source == 'kaggle':
        db_path = f'./databases/{database}/{database}.sqlite'
    elif source == 'bird':
        db_path = f'./bird_dev/dev_databases/{database}/{database}.sqlite'
        
    if not os.path.isfile(db_path):
        print("cannot find file")
        return False
    results = ''
    try:
        # Connect to sqlite db
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        results = cursor.execute(sql).fetchall()
    # If we hit an error, that's not a match I guess...
    except KeyboardInterrupt:
        cursor.close()
        connection.close()
        print("KeyboardInterrupt")
        return False
    except Exception as ex:
        cursor.close()
        connection.close()
        print(ex)
        return False
    finally:
        cursor.close()
        connection.close()
    return results

In [6]:
def GPT4_turbo_generation(prompt, t = 0.0, p = 0.0, stopWord="Answer:",):
    response = client.chat.completions.create(
        model = 'gpt-4-turbo-preview',
        messages=[{"role": "user", "content": prompt}],
        n = 1,
        stream = False,
        temperature=t,
        max_tokens=4000,
        top_p = p,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        logprobs=True,
        stop = [stopWord]
    )
    logprobs = [token.logprob for token in response.choices[0].logprobs.content]
    perplexity_score = np.exp(-np.mean(logprobs))
    return response.choices[0].message.content.strip(), perplexity_score

def GPT35_generation(prompt, t = 0.0, p=0.0, fp=0.0, pp=0.0, stopWord="Answer:",):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}],
        n = 1,
        stream = False,
        temperature=t,
        max_tokens=4000,
        top_p = p,
        frequency_penalty=fp,
        presence_penalty=pp,
        logprobs=True,
        stop = [stopWord]
    )
    logprobs = [token.logprob for token in response.choices[0].logprobs.content]
    perplexity_score = np.exp(-np.mean(logprobs))
    return response.choices[0].message.content.strip(), perplexity_score

In [7]:
df = pd.read_csv('kaggle_dataset.csv')
userstudy = []
with open('./logs/user_study.pkl', 'rb') as f:
    userstudy = pickle.load(f)
survey_questions = []
# add original gold query inside
for d in userstudy:
    if 'Question2Ask' in d:
        assert len(d['Question2Ask']) == len(d['Answer2Question']), print(d['nl'])
    q = d['nl']
    sql = df.loc[df['nl'] == q]['sql'].values
    d['gold'] = sql[0]
    d["target_schema"] = df.loc[df['nl'] == q]['target_schema'].values
    survey_questions.append(q)

print(len(df))
# drop the user study questions
df = df[~df['nl'].isin(survey_questions)]
print(len(df))

272
208


In [8]:
df = df.reset_index()

In [9]:
bird_data_df = pd.read_json('sampled_bird_dev.json')
bird_data_df.reset_index(level=0, inplace=True)
bird_train_df = pd.read_json('bird_train.json')

In [10]:
feedback_prefix_v1='''Example Natural Language Question:```'How many acres burned in fires in California each year between 2000 and 2005?'```
Example Gold Query that answers Natural Language Question:```'SELECT\n  SUM(FIRE_SIZE),\n  FIRE_YEAR\nFROM Fires\nWHERE\n  State = "CA" AND FIRE_YEAR BETWEEN 2000 AND 2005\nGROUP BY\n  FIRE_YEAR'```
Example Clarification Question:```What information should the output table contain? a) output table has two columns: the total acres burned and the year, b) output table has one column: the total acres burned for each year, c) output table has one column: the total acres burned across all target years, d) other (please specify).```
Example Reasoning:```The clarification question is asking what information should the output table contain. The fields in the output table are determined by the SELECT clause in the gold query. The gold query uses ‘SELECT  SUM(FIRE_SIZE), FIRE_YEAR’ to select the outputs. As a result, the output table has two columns, the total acres burned and the year. Hence, choice a is correct.```
Example Answer:```a) output table has two columns: the total acres burned and the year```

Example Natural Language Question:```'Which states had the largest number of fires in 2001?’```
Example Gold Query that answers Natural Language Question:```SELECT\n  State\nFROM Fires\nWHERE\n  FIRE_YEAR = 2001\nGROUP BY\n  State\nORDER BY\n  COUNT(*) DESC\nLIMIT 1```
Example Clarification Question:```Is the largest number of fires referring to? a) the total size of all fire incidents, b) the number of fire incidents, c) the largest size of all fire incidents, d) other (please specify).```
Example Reasoning:```The clarification question is asking about how to represent the largest number of fires. The gold query uses ‘ORDER BY COUNT(*) DESC LIMIT 1’ to find the largest number of fires. As a result, choice a is correct.```
Example Answer:```b) the number of fire incidents```

Example Natural Language Question:```What was the most common cause of fire between 2000 and 2005?```
Example Gold Query that answers Natural Language Question:```'SELECT\n  STAT_CAUSE_DESCR\nFROM Fires\nWHERE\n  FIRE_YEAR BETWEEN 2000 AND 2005\nGROUP BY\n  STAT_CAUSE_DESCR\nORDER BY\n  COUNT(*) DESC\nLIMIT 1'```
Example Clarification Question:```Which information should be used to represent the 'cause of fire'? a) the code that represents the cause, b) the description of the cause, c) both the code and the description of the cause, d) other (please specify).```
Example Reasoning:```The clarification question is asking for which column should be used to represent the cause of fire. The gold query uses the STAT_CAUSE_DESCR to represent the cause. STAT_CAUSE_DESCR column contains information about the description of the causes. As a result, choice b is correct.```
Example Answer:```b) the description of the cause```

Example Natural Language Question:```What was the cause of the largest wildfire in Utah in the year 1997?```
Example Gold Query that answers Natural Language Question:```SELECT\n  *\nFROM Fires\nWHERE\n  State = "UT" AND FIRE_YEAR = 1997\nORDER BY\n  FIRE_SIZE DESC\nLIMIT 1```
Example Clarification Question:```Is the cause of fire referring to? a) the code that represents the cause, b) the description of the cause, c) both the code and the description of the cause, d) other (please specify).'```
Example Reasoning:```The clarification question is asking for what columns represent the cause of fire. The natural language question is asking for the causes and the gold query is selecting all fields in the database to the output table. Hence, based on the gold query, the cause of the fire is represented by all fields from the Fires table. As a result, the correct answer is d, since a, b and c are all wrong.```
Example Answer:```d) other (please specify). Please use all fields in the fires table to represent the 'cause' information.```

Example Natural Language Question:```Whose CDs sells best?```
Example Gold Query that answers Natural Language Question:```'SELECT\n  artist\nFROM torrents\nGROUP BY\n  artist\nORDER BY\n  SUM(totalSnatched) DESC\nLIMIT 1'```
Example Clarification Question:```Which column should be used to identify music related to 'CD'? a) groupName, b) tag, c) releaseType, d) other (please specify)```
Example Reasoning:```The clarification question is asking for what columns represent the CDs. The gold query does not use a WHERE clause to filter the CDs. Hence, the CD information is not contained in the tag column or the release type column. As a result, choice a, b, and c are all wrong.```
Example Answer:```d) other (please specify). Consider all music. No 'CD' filters on group name or tag or release type.```
'''

feedback_v2 = """Your task is to answer the multiple choice clarification question truthfully based on the Gold Query.

Natural Language Question: ```{nlq}```
Gold Query: ```{query}```
The gold query is the sql answer to the natural language question

Multiple Choice Clarification Question: ```{question}```

Answer the above clarification question truthfully based on the Gold Query.

First, reason which portion of the gold query answers the clarification question.
Next, consider the correctness of each multiple choice answers based only on the gold query.
Lastly, output the answer with format: answer_to_cq = "".
If none of the multiple choices are correct or you selected `other (please specify)`, give a short answer to the clarification question after `other (please specify)`.


Let's think step by step:
"""

In [15]:
cq_prefix_v1 = '''/* some examples are provided before the main question */\n\n
Example Question: ```Which artist/group is most productive?```
Example Reasoning:```Based on the question 'Which artist/group is most productive?', it is clear that the SQL answer should contain the 'most productive artist/group’. Hence, the SQL answer needs to contain ORDER BY and LIMIT 1. However, the question itself is ambiguous, and how productivity is determined needs to be clarified. Sorting based on the number of music produced or the sum of total downloads are both valid choices. In addition, it is unclear what columns should be used to represent the 'artist/group'. The artist and the groupName columns contain information about 'artist/group'.```
Example Ambiguity Type:```[‘AmbQuestion’, ‘AmbColumn’]```
Example Multiple Choice Clarification Question:```How to rank artist/group productivity? a) rank by the number of records produced, b) rank by the total number of downloads, c) other (please specify).
Which columns represent the 'artist/group' information? a) the artist column only, b) the groupName column only, c) both the artist column and the groupName column, d) other (please specify).```

Example Question: ```Which Premier League matches ended in a draw in 2016?```
Example Reasoning:```It is clear that the SQL answer to this question needs to contain a WHERE clause for three conditions: 'Premier League', 'draw', and 'in 2016'. However, it needs to be clarified i) which column should be used in the WHERE clause and ii) what the predicate value should be used. For instance, to find 2016, it is unclear if the SQL answer should use the season column or the DateTime column. Also, there are several choices for the predicate value, such as '2016', '2016/2017', or '2015/2016'. Moreover, the output table is unclear. The question did not specify what fields should be contained in the output table.```
Example Ambiguity Type:```['AmbColumn', 'AmbValue', 'AmbOutput']```
Example Multiple Choice Clarification Question:```Is the year '2016' referring to? a) season is 2016, b) season is either 2015/2016 or 2016/2017, c) the date time is at year 2016, d) other (specify).
How to find the 'Premier league'? a) consider all leagues, b) consider only the league with name 'Premier League', c) consider only the league that contains the name ‘Premier’, d) other (specify).
What fields represent the target 'matches'? a) all fields from football data table, b) a single column with league information, c) other (specify).```

Example Question: ```Which type of crime has the highest rate of ‘Investigation complete’?```
Example Reasoning:```It is clear that the SQL answer to this question needs to contain a WHERE clause to find crimes that have 'Investigation complete' outcomes, uses ORDER BY and LIMIT 1 to find the type of crime with the highest rate, and the output table has only one row. However, it needs to be clarified i) what predicate value should be used for 'Investigation complete', and ii) how to represent the 'rate', and iii) if the output table contains only the crime type column or the crime type column with the highest rate aggregate.```
Example Ambiguity Type:```['AmbQuestion', 'AmbValue', ‘AmbOutput’]```
Example Multiple Choice Clarification Question:```What information should be used to find 'Investigation complete'? a) see if outcome contains the phrase 'Investigation complete', b)  see if outcome is 'Investigation complete; no suspect identified', c) other (please specify).
How to represent the 'rate' for each crime type? a) the number of crimes for each crime type over all crimes, b) a) the number of crimes for each crime type over the all crimes that are 'Investigation Complete', c) the number of crimes for each crime type, c) other (please specify).
What fields should the output table contain? a) two fields: the crime type and the rate, b) one field: the crime type only, c) other (please specify).```

Example Question: ```For award winners, which position that has the most hall of fame players?```
Example Reasoning:```It is clear that the 'award winners' information is contained in the player_award table, the 'hall of fame players' are players whose induction is 'Y' in the hall_of_fame table, and the output table only has a single row. However, it is unclear which column contains the position information. In addition, the question needs to clarify if the output should contain only the position information or the position information along with the number of hall-of-fame players.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbOutput’]```
Example Multiple Choice Clarification Question:```How should the 'position' for players be identified? a) by the 'award_id' column, b) by the 'category' column, c) by the 'notes' column, d) other (please specify).
What fields should be contained in the output table? a) one field: the position, b) two fields: the position and the number of hall-of-fame players, c) other (please specify).```

Example Question: ```How many Wisconsin school districts receive federal funding?```
Example Reasoning:```The output should contain only the number of Wisconsin school districts that receive federal funding. However, it is unclear which column and predicate values should be used to find 'Wisconsin school districts' and how to determine if a district has 'received federal funding'.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’]```
Example Multiple Choice Clarification Question:```
Is 'Wisconsin school districts' referring to? a) all school districts in the state Wisconsin, b) school districts with names that contain Wisconsin, c) other (please specify).
How should the answer determine if a district has received federal funding? a) based on the t_fed_rev is larger than 0, b) the answer does not need to consider this aspect, c) other (please specify).```

Example Question: ```How many students from the ages of 5 to 17 are enrolled at the State Special School school in Fremont for the 2014-2015 academic year?```
Example Reasoning:```It is clear that the phrase 'students from the ages of 5 to 17 are enrolled
' from the question refers to the `Enrollment (Ages 5-17)` column in the frpm table, and the phrase '2014-2015 academic year' filters the result based on `Academic Year` column in the frpm table. However, it needs to be clarified which columns should be used to find results related to the 'State Special School school' and 'Fremont', and what predicate values should be used to represent 'State Special School school' and 'Fremont'.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’]```
Example Multiple Choice Clarification Question:```
Which column contains the information about 'State Special School'? a) `School Type` in the frpm table, b) `EdOpsCode` in the schools table, c) `EILCode` in the schools table, d) other (please specify).
Which column contains the information about 'Fremont'? a) `City` column in the schools table, b) `County Name` in the frpm table, c) `District Name`  name in the frpm table, d) other (please specify).
What predicate value should be used to represent 'State Special School'? a) an exact string match on 'SSS', b) any name like '%State Special%', c) an exact string match on 'State Special School', d) other (please specify).```

Example Question: ```What is the average number of crimes committed in 1995 in regions where the number exceeds 4000 and the region has accounts that are opened starting from the year 1997?```
Example Reasoning:```It is clear that the output table contains the average number of crimes committed in 1995, and there needs to be a where clause to filter the year based on '1997' and filter the number of crimes committed in 1995 with '4000'. However, it needs to be clarified which columns represent the number of crimes committed in 1995 and how to find years after '1997'.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’]```
Example Multiple Choice Clarification Question:```
Which column contains the information about the 'number of crimes committed in 1995'? a) `A15` column in the district table, b) `A5` column in the district table, c) other (please specify).
How do I filter the account based on the year 1997? a) extract the year from the `date` column in the account table and then find the year larger or equal to 1997, b) similar to a) except use the trans
instead of account table, c) similar to a) except use the loan table instead of account table, d) other (please specify).```

Example Question: ```How many molecules have a double bond type and among these molecule, how many are labeled as carcinogenic compounds?```
Example Reasoning:```The output table clearly has two columns: the number of unique molecules and the number of carcinogenic compounds. The results should be filtered based on the 'double bound type' and 'carcinogenic compound'. However, it needs to be clarified on which column and what predicate value represents the 'double bond type' and 'carcinogenic compound'. ```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’]```
Example Multiple Choice Clarification Question:```How to represent the double bound type? a) double bound type is identified based on the `bond_type` column in the bond table using predicate value '='
, b) double bound type is identified based on the `bond_type` column in the bond table using predicate value 'double', c) other (please specify)
How to find 'carcinogenic compound'? a) it can be identified using the `label` column in the molecule table with predicate value '+', b) it can be identified using the `label` column in the molecule table with predicate value '++', c) other (please specify)```
Example Question: ```What is the status of card "Cloudchaser Eagle"?```
Example Reasoning:```It is clear that the "Cloudchaser Eagle" should be used to identify cards, and the status refers to the `status` column in the legalities table. However, what should be included in the output table and which column should be used to identify the "Cloudchaser Eagle" need further clarification.```
Example Ambiguity Type:```[‘AmbOutput’, ‘AmbColumn’’]```
Example Multiple Choice Clarification Question:```
What information should be included in the output table? a) the output table has one column with the unique statuses, b) the output table has one column with all statuses including duplicates, c) the output table has two columns with the unique statuses and the card name, d) other (please specify).
How to find the "Cloudchaser Eagle" cards? a) exact string match of "Cloudchaser Eagle" on the `name` column in the cards table, b) pattern string match of "%Cloudchaser Eagle%" on the `originalText` column in the cards table, c) exact string match of "Cloudchaser Eagle" on the `keywords` column in the cards table, d)other (please specify).
```
Example Question: ```For patients born between Year 1930 and 1940, how many percent of them were inpatient?```
Example Reasoning:```​​It is clear that the output table has a single column with the percentage information, and the where clause should include a filter based on the `birthday` column from the Patient table. However, finding the 'inpatient' and how to represent the percentage in the output table need to be further clarified.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’, ‘AmOutput’]```
Example Multiple Choice Clarification Question:```How to find the 'inpatient' information? a) 'inpatient' refers to having an exact match of '+' in the `Admission` column from the Patient table, b) inpatient refers to having a partial match of '%inpatient%' in the `Description` column from the Patient table, c) other (please specify).
How to format the percentage in the output table? a) the percentage is calculated as the number of inpatients divided by the number of patients times 100, b) the percentage is calculated as the number of inpatients divided by the number of patients, c) other (please specify).
```
'''

SRA = """Your task is to ask the user a single multiple choice clarification question to help you find the correct SQL query.
User does not understand SQL. The clarification question should be comprehensive by people with no coding experience.

QUESTION: ```{question}```
DATABASE SCHEMA:```{schema}```

Previous Clarification Questions and User Feedback: ```{cqs}```

List of Incorrect Queries: ```{sqls}```
The above SQL Queries are wrong.

Here are four ambiguity categories to consider:
    - AmbQuestion: Is the question itself ambiguous?
    - AmbTableColumn: Is there ambiguity in mapping the entities from QUESTION to tables and columns in DATABASE SCHEMA?
    - AmbOutput: What fields and how many fields should be contained in the output table?
    - AmbValue: What predicate value should be used to filter results?

Let's think step by step.

STEP 1, Summarize the information that is clear based on the ansewrs to previous clarification questions and incorrect queries.

STEP 2, Evaluate whether AmbQuestion, AmbColumn, AmbOutput, and AmbValue remain in formulating a SQL query to answer the QUESTION, considering each category individually.

STEP 3, Ask a multiple-choice clarification question to clarify the remaining ambiguities and help you find the correct SQL query. Ensure it's simple enough for someone without coding experience to understand.
        Use format: mul_choice_cq = "".
        
Answer:
"""

In [16]:
sql_generation = '''Complete sqlite SQL query only and with no explanation.
Your task is to write a SQLITE SQL QUERY that correctly answers the following QUESTION and align with database schema.

QUESTION: {question}

The DATABASE SCHEMA: {schema}

Answer:'''


sql_generation_v2 = '''Complete a SQLite SQL query only with no explanation.
Your task is to complete a new SQLite SQL QUERY that correctly answers the USER QUESTION, aligns with DATABASE SCHEMA, and follows USER FEEDBACK.

USER QUESTION: ```{question}```
The DATABASE SCHEMA: ```{schema}```

The user has provided a list of incorrect SQL queries and the following sql queries are wrong.
Incorrect Queries: ```[{sqls}]``` 

The user has provided some feedback. User Feedback are the golden truth.
User Feedback: {cqas}

You know the Incorrect Queries are wrong. Do not rewrite the exact same incorrect queries.
Please listen carefully to the user feedback. Write the new correct query.

Answer:
'''



fix_invalid_v1 = """Your task is to fix the invalid SQL query.

Database schema```{schema}```

Invalid SQL Query: ```{sql}```
The above SQL query is not valid and can not be executed.
Exception Message:```{ex}```

Fix the errors and output a new SQL query that is executable and align with Database schema.

Write the new executable SQL query in the format: sql = ``` ```.

Answer:
"""


In [17]:
selfdebug_examples_prefix = '''Example Question: ```In which year were most departments established?```
Example Incorrect Queries: ```SELECT creation, COUNT(*) FROM department GROUP BY creation ORDER BY
COUNT(*) DESC LIMIT 1```
Feedback: ```The above SQL predictions are wrong. Please fix the SQL.```
Example Correct Query: ```SELECT creation FROM department GROUP BY creation ORDER BY COUNT(*) DESC LIMIT 1```
-------
Example Question: ```Which customers have both "On Road" and "Shipped" as order status? List the customer names.```
Example Incorrect Queries: ```SELECT customers.customer_name FROM customers JOIN orders ON customers.customer_id = orders.customer_id WHERE orders.order_status = "On Road" AND orders.order_status = "Shipped"```
Feedback: ```The above SQL predictions are wrong. Please fix the SQL.```
Example Correct Query: ```SELECT customers.customer_name FROM customers JOIN orders ON customers.customer_id = orders.customer_id WHERE orders.order_status = "On Road" 
INTERSECT SELECT customers.customer_name FROM customers JOIN orders ON customers.customer_id = orders.customer_id WHERE orders.order_status = "Shipped"```
-------
Example Question: ```Show origins of all flights with destination Honolulu.```
Example Incorrect Queries: ```SELECT origin FROM flight WHERE destination = "HONO"```
Feedback: ```The above SQL predictions are wrong. Please fix the SQL.```
Example Correct Query: ```SELECT origin FROM flight WHERE destination = "Honolulu"```
-------
Example Question: ```What is the average longitude of stations that never had bike availability more than 10?```
Example Incorrect Queries: ```SELECT AVG(long) FROM station WHERE id IN (SELECT station_id FROM
status WHERE bikes_available <= 10)```
Feedback: ```The above SQL predictions are wrong. Please fix the SQL.```
Example Correct Query: ```SELECT AVG(long) FROM station WHERE id NOT IN (SELECT station_id FROM
status WHERE bikes_available > 10)```
-------
Example Question: ```Show the name and the nationality of the oldest host.```
Example Incorrect Queries: ```SELECT name, nationality FROM host WHERE age = (SELECT MIN(age) FROM
host)```
Feedback: ```The above SQL predictions are wrong. Please fix the SQL.```
Example Correct Query: ```SELECT name, nationality FROM host ORDER BY age DESC LIMIT 1```
-------
Example Question: ```How many different statuses do cities have?```
Example Incorrect Queries: ```SELECT COUNT(status) FROM city```
Feedback: ```The above SQL predictions are wrong. Please fix the SQL.```
Example Correct Query: ```SELECT COUNT(DISTINCT status) FROM cit```
-------'''
selfdebug_examples = selfdebug_examples_prefix.split('-------')

selfdebug_few_shot = []
for i in range(1,7):
    prefix = []
    for j in range(i):
        prefix.append(selfdebug_examples[j])
    selfdebug_few_shot.append('\n'.join(prefix))

sql_generation_selfdebug = '''Complete a SQLite SQL query only with no explanation.
Your task is to complete a new SQLite SQL QUERY that correctly answers the USER QUESTION, aligns with DATABASE SCHEMA, and follows USER FEEDBACK.

USER QUESTION: ```{question}```
The DATABASE SCHEMA: ```{schema}```

Incorrect Queries: ```[{sqls}]```
The above SQL predictions are wrong. Please fix the SQL.
Write the new correct query.

Answer:
'''


In [18]:
fewshot_prefix = "/* some examples are provided before the main question */\n\n"

In [19]:
embeddings = OpenAIEmbeddings()
userstudy_vectorstore = Chroma(persist_directory="./chroma_db_userstudy", embedding_function=embeddings)
bird_dev_vectorstore = Chroma(persist_directory="./chroma_db_birddev", embedding_function=embeddings)

In [22]:
def baselineFewShot(data_frame, history_log, log_name, rounds, num_of_tests, model_name, vectorstore, num_examples, data_source):
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    assert num_examples < len(selfdebug_few_shot)
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    
    
    example_selector = SemanticSimilarityExampleSelector(
        vectorstore=vectorstore,
        k=num_examples,
    )
    example_prompt = PromptTemplate(
        input_variables=['nl', 'gold'],
        template="\nExample Question: {nl}\nExample Answer: {gold}\n",
    )
    sql_generation_few_shot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        suffix=sql_generation,
        input_variables=["question", "schema"],
    )
    
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
        history_log[index] = {}
        history_log[index]['sql_log'] = []
        history_log[index]['cq_log'] = []
        history_log[index]['feedback_log'] = []
        order = 0
        
        d = data_frame.iloc[[index]] 
        gold = d['sql'].values[0]
        dbname = d['target_db'].values[0]
        nlq = d['nl'].values[0]
        dbschema = d['target_schema'].values[0]
        cqs_and_answers = []
        query = set()
        print("nl: ", nlq, index)
        print("gold: ", gold)
        
        sql_prompt = ''
        
        sql_prompt = sql_generation_few_shot_prompt.format(schema=dbschema, question=nlq)
        
        sql_prompt = fewshot_prefix + sql_prompt
        sql_query, pscore= generation(sql_prompt)
        history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
        order += 1
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query, pscore)
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=d['nl'],\
                                                   sql=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            print("After Fix Invalid SQL: ", sql, pscore)
            valid_sql = clean_query(sql.split("sql = ")[-1])
            history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
            order += 1
            query.add(valid_sql)
            execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            sql_prompt = sql_generation_selfdebug.format(schema=dbschema, question=nlq,\
                                              sqls=",\n".join(query))
            sql_prompt = fewshot_prefix + selfdebug_few_shot[num_examples-1] + sql_prompt
            sql_query, pscore= generation(sql_prompt)
            print("sql: ", sql_query, pscore)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            sql_query = clean_query(sql_query)
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                     sql=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                print("After Fix Invalid SQL: ", sql, pscore)
                valid_sql = clean_query(sql.split("sql = ")[-1])
                query.add(valid_sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
                order += 1
                execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)

In [23]:
def askClarificationQuestions(data_frame, history_log, log_name, rounds, num_of_tests, model_name, vectorstore, vectorstore_feedback, num_examples, data_source, baseline_history):    
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    
    example_selector_ini = SemanticSimilarityExampleSelector(
        vectorstore=vectorstore,
        k=num_examples,
    )
    feedback_example_selector = SemanticSimilarityExampleSelector(
        vectorstore=vectorstore_feedback,
        k=num_examples,
    )
    
    example_prompt = PromptTemplate(
        input_variables=['nl', 'gold'],
        template="\nExample Question: {nl}\nExample Answer: {gold}\n",
    )
    sql_generation_few_shot_prompt = FewShotPromptTemplate(
        example_selector=example_selector_ini,
        example_prompt=example_prompt,
        suffix=sql_generation,
        input_variables=["question", "schema"],
    )
    feedback_example_prompt = PromptTemplate(
        input_variables=['nl', 'gold', 'feedback'],
        template="\nExample Question: {nl}\nExample Feedback:{feedback}\nExample Answer: {gold}",
    )

    sql_generation_feedback_few_shot_prompt = FewShotPromptTemplate(
        example_selector=feedback_example_selector,
        example_prompt=feedback_example_prompt ,
        suffix=sql_generation_v2,
        input_variables=["question", "schema", "sqls", "cqas"],
    )
    
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
        if index in baseline_history and baseline_history[index]["num_cq_asked"] == 0:
            history_log[index] = baseline_history[index].copy()
            continue
        
        history_log[index] = {}
        history_log[index]['sql_log'] = []
        history_log[index]['cq_log'] = []
        history_log[index]['feedback_log'] = []
        order = 0
        
        d = data_frame.iloc[[index]] 
        gold = d['sql'].values[0]
        dbname = d['target_db'].values[0]
        nlq = d['nl'].values[0]
        dbschema = d['target_schema'].values[0]
        cqs_and_answers = []
        query = set()
        print("nl: ", nlq, index)
        print("gold: ", gold)

        sql_prompt = ''
        sql_prompt = sql_generation_few_shot_prompt.format(schema=dbschema, question=nlq)
        sql_prompt = fewshot_prefix + sql_prompt  
        sql_query, pscore= generation(sql_prompt)
        history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
        order += 1
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query, pscore)
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=d['nl'],\
                                                 sql=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            print("After Fix Invalid: ", sql, pscore)
            valid_sql = clean_query(sql.split("sql = ")[-1])
            history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
            order += 1
            query.add(valid_sql)
            execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "Answer: "+cqs_and_answers[i]
                
            cq_prompt = SRA.format(schema=dbschema, question=nlq,\
                                            sqls=",\n".join(query), cqs=cqas)
            cq_prompt = cq_prefix_v1 + cq_prompt
            cq, pscore= generation(cq_prompt)
            history_log[index]['cq_log'].append((order, cq_prompt, cq, pscore))
            order += 1
            print("cq: ", cq, pscore)
            if "mul_choice_cq = " in cq:
                cq = cq.split("mul_choice_cq = ")[-1]
            
            feedback_prompt = feedback_v2.format(query = gold, question = cq, nlq=nlq)
            feedback_prompt = fewshot_prefix + feedback_prefix_v1 + feedback_prompt
            feedback, pscore= GPT4_turbo_generation(feedback_prompt, stopWord="Let's think step by step:")
            print()
            print("feedback, ", feedback, pscore)
            history_log[index]['feedback_log'].append((order, feedback_prompt, feedback, pscore))
            order += 1
            if "answer_to_cq =" in feedback:
                feedback = feedback.split("answer_to_cq =")[-1]
            cqs_and_answers.append(cq)
            cqs_and_answers.append(feedback)
        
            # fix incorrect sql based on user feedback
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "User Feedback: "+cqs_and_answers[i] 
            
            # use examples from the user study
            sql_prompt = sql_generation_feedback_few_shot_prompt.format(schema=dbschema, question=nlq,\
                                              sqls="\n".join(query), cqas=cqas)
            sql_prompt = fewshot_prefix + sql_prompt
            sql_query, pscore= generation(sql_prompt)
            print("sql: ", sql_query, pscore)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            sql_query = clean_query(sql_query)
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                       sql=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                print("Afrer Fix Invalid: ", sql, pscore)
                valid_sql = clean_query(sql.split("sql = ")[-1])
                query.add(valid_sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
                order += 1
                execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
                
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)

In [24]:
# decide when to stop
SRA_ES = """Your task is to identify the remaining ambiguity.
If there are remaining ambiguities, then ask the user a single multiple choice clarification question to help you find the correct SQL query.
User does not understand SQL. The clarification question should be comprehensive by people with no coding experience.

QUESTION: ```{question}```
DATABASE SCHEMA:```{schema}```

Previous Clarification Questions and User Feedback: ```{cqs}```

List of Incorrect Queries: ```{sqls}```
The above SQL Queries are wrong.

Here are four ambiguity categories to consider:
    - AmbQuestion: Is the question itself ambiguous?
    - AmbTableColumn: Is there ambiguity in mapping the entities from QUESTION to tables and columns in DATABASE SCHEMA?
    - AmbOutput: What fields and how many fields should be contained in the output table?
    - AmbValue: What predicate value should be used to filter results?

Let's think step by step.

STEP 1, Summarize the information that is clear based on the ansewrs to previous clarification questions.

STEP 2, Evaluate whether AmbQuestion, AmbColumn, AmbOutput, and AmbValue remain in formulating a SQL query to correctly answer the QUESTION, considering each category individually.

STEP 3, If no remaining ambiguities are identified, then output "NO AMBIGUITY".
        Otherwise, ask a multiple-choice clarification question to clarify the remaining ambiguities and help you find the correct SQL query. Ensure it's simple enough for someone without coding experience to understand. Use format: mul_choice_cq = "".
        
Answer:
"""

In [25]:
def askCQsBreakNoAmb(data_frame, history_log, log_name, rounds, num_of_tests, model_name, vectorstore, vectorstore_feedback, num_examples, data_source, baseline_history):    
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    
    example_selector_ini = SemanticSimilarityExampleSelector(
        vectorstore=vectorstore,
        k=num_examples,
    )
    feedback_example_selector = SemanticSimilarityExampleSelector(
        vectorstore=vectorstore_feedback,
        k=num_examples,
    )
    
    example_prompt = PromptTemplate(
        input_variables=['nl', 'gold'],
        template="\nExample Question: {nl}\nExample Answer: {gold}\n",
    )
    sql_generation_few_shot_prompt = FewShotPromptTemplate(
        example_selector=example_selector_ini,
        example_prompt=example_prompt,
        suffix=sql_generation,
        input_variables=["question", "schema"],
    )
    feedback_example_prompt = PromptTemplate(
        input_variables=['nl', 'gold', 'feedback'],
        template="\nExample Question: {nl}\nExample Feedback:{feedback}\nExample Answer: {gold}",
    )

    sql_generation_feedback_few_shot_prompt = FewShotPromptTemplate(
        example_selector=feedback_example_selector,
        example_prompt=feedback_example_prompt ,
        suffix=sql_generation_v2,
        input_variables=["question", "schema", "sqls", "cqas"],
    )
    
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
        history_log[index] = {}
        history_log[index]['sql_log'] = []
        history_log[index]['cq_log'] = []
        history_log[index]['feedback_log'] = []
        order = 0
        d = data_frame.iloc[[index]] 
        gold = d['sql'].values[0]
        dbname = d['target_db'].values[0]
        nlq = d['nl'].values[0]
        dbschema = d['target_schema'].values[0]
        cqs_and_answers = []
        query = set()
        print("nl: ", nlq, index)
        print("gold: ", gold)
        sql_query = ''
        if index in baseline_history and 'sql_log' in baseline_history[index]:
            history_log[index]['sql_log'].append(baseline_history[index]['sql_log'][0])
            order, sql_prompt, sql_query, pscore = history_log[index]['sql_log'][0]
        else:
            sql_prompt = ''
            sql_prompt = sql_generation_few_shot_prompt.format(schema=dbschema, question=nlq)
            sql_prompt = fewshot_prefix + sql_prompt  
            sql_query, pscore= generation(sql_prompt)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query, pscore)
        order += 1
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=d['nl'],\
                                                 sql=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            print("After Fix Invalid: ", sql, pscore)
            valid_sql = clean_query(sql.split("sql = ")[-1])
            history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
            order += 1
            query.add(valid_sql)
            execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "Answer: "+cqs_and_answers[i]
                
            cq_prompt = SRA_ES.format(schema=dbschema, question=nlq,\
                                            sqls=",\n".join(query), cqs=cqas)
            cq_prompt = cq_prefix_v1 + cq_prompt
            cq, pscore= generation(cq_prompt)
            history_log[index]['cq_log'].append((order, cq_prompt, cq, pscore))
            order += 1
            print("cq: ", cq, pscore)
            if "NO AMBIGUITY" in cq:
                print()
                print("-----NO AMBGUITY-----")
                print()
                break
            if "mul_choice_cq = " in cq:
                cq = cq.split("mul_choice_cq = ")[-1]
            
            feedback_prompt = feedback_v2.format(query = gold, question = cq, nlq=nlq)
            feedback_prompt = fewshot_prefix + feedback_prefix_v1 + feedback_prompt
            feedback, pscore= GPT4_turbo_generation(feedback_prompt, stopWord="Let's think step by step:")
            print()
            print("feedback, ", feedback, pscore)
            history_log[index]['feedback_log'].append((order, feedback_prompt, feedback, pscore))
            order += 1
            if "answer_to_cq =" in feedback:
                feedback = feedback.split("answer_to_cq =")[-1]
            cqs_and_answers.append(cq)
            cqs_and_answers.append(feedback)
        
            # fix incorrect sql based on user feedback
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "User Feedback: "+cqs_and_answers[i] 
            
            # use examples from the user study
            sql_prompt = sql_generation_feedback_few_shot_prompt.format(schema=dbschema, question=nlq,\
                                              sqls="\n".join(query), cqas=cqas)
            sql_prompt = fewshot_prefix + sql_prompt
            sql_query, pscore= generation(sql_prompt)
            print("sql: ", sql_query, pscore)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            sql_query = clean_query(sql_query)
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                       sql=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                print("Afrer Fix Invalid: ", sql, pscore)
                valid_sql = clean_query(sql.split("sql = ")[-1])
                query.add(valid_sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
                order += 1
                execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
                
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)

## Put Together

## 5 shot gpt3.5

In [153]:
# kaggel_baseline_5shot_histories = {}
# bird_baseline_5shot_histories = {}
# bird_interaction_5shot_histories = {}
# kaggle_interaction_5shot_histories = {}
# kaggle_interaction_5shot_histories_stop_early = {}
# bird_interaction_5shot_histories_stop_early = {}

baselineFewShot(df, kaggel_baseline_5shot_histories,\
                log_name='./fewshotlogs/kaggle_gpt35_baseline_4round_5shot.pkl', rounds=4,\
                num_of_tests=208, model_name='gpt35turbo', vectorstore=userstudy_vectorstore,\
                num_examples=5, data_source='kaggle')

baselineFewShot(bird_data_df, bird_baseline_5shot_histories,\
                log_name='./fewshotlogs/bird_gpt35_baseline_4round_5shot_from_devvect.pkl', rounds=4,\
                num_of_tests=176, model_name='gpt35turbo', vectorstore=bird_dev_vectorstore,\
                num_examples=5, data_source='bird')


askClarificationQuestions(data_frame=df, history_log=kaggle_interaction_5shot_histories,\
                  log_name='./fewshotlogs/kaggle_gpt35_interaction_4round_5shot.pkl', rounds=4,\
                  num_of_tests=208, model_name='gpt35turbo', vectorstore=userstudy_vectorstore, vectorstore_feedback=userstudy_vectorstore,\
                num_examples=5, data_source='kaggle', baseline_history=kaggel_baseline_5shot_histories)


askClarificationQuestions(data_frame=bird_data_df, history_log=bird_interaction_5shot_histories,\
                  log_name='./fewshotlogs/bird_gpt35_interaction_4round_5shot_from_devvect.pkl', rounds=4,\
                  num_of_tests=176, model_name='gpt35turbo', vectorstore=bird_dev_vectorstore, vectorstore_feedback=userstudy_vectorstore,\
                num_examples=5, data_source='bird', baseline_history=bird_baseline_5shot_histories)

askCQsBreakNoAmb(data_frame=df, history_log=kaggle_interaction_5shot_histories_stop_early,\
                  log_name='./fewshotlogs/kaggle_gpt35_interaction_4round_5shot_stop_early.pkl', rounds=4,\
                  num_of_tests=208, model_name='gpt35turbo', vectorstore=userstudy_vectorstore, vectorstore_feedback=userstudy_vectorstore,\
                num_examples=5, data_source='kaggle', baseline_history=kaggle_interaction_5shot_histories )

askCQsBreakNoAmb(data_frame=bird_data_df, history_log=bird_interaction_5shot_histories_stop_early,\
                  log_name='./fewshotlogs/bird_gpt35_interaction_4round_5shot_from_devvec_stop_early.pkl', rounds=4,\
                  num_of_tests=176, model_name='gpt35turbo', vectorstore=bird_dev_vectorstore, vectorstore_feedback=userstudy_vectorstore,\
                num_examples=5, data_source='bird', baseline_history=bird_interaction_5shot_histories)

nl:  Name the most popular release on houston. 0
gold:  SELECT
  T2.groupName
FROM torrents AS T2
JOIN tags AS T1
  ON T1.id = T2.id
WHERE
  T1.tag = "houston"
ORDER BY
  totalSnatched DESC
LIMIT 1
sql:  SELECT
  groupName
FROM torrents
WHERE
  artist = "houston"
ORDER BY
  totalSnatched DESC
LIMIT 1 1.002141542795228
in execute query
cq:  STEP 1: 
Based on the previous clarification questions and incorrect queries, it is clear that the question is asking for the most popular release on 'houston'. The database schema contains tables for 'torrents' and 'tags', with columns like 'groupName', 'totalSnatched', 'artist', and 'releaseType'.

STEP 2:
AmbQuestion: The question itself is ambiguous as it does not specify how to define 'popular release'.
AmbColumn: It is unclear which column should be used to represent 'houston'.
AmbOutput: It is unclear what fields should be contained in the output table.
AmbValue: It is unclear what predicate value should be used to filter the results.

STEP 3:

sql:  SELECT
  groupName
FROM torrents
WHERE
  artist = "houston"
ORDER BY
  totalSnatched DESC
LIMIT 1 1.0051429876426383
in execute query

------next question------

nl:  Which albums have been downloaded more than 100 times? 1
gold:  SELECT DISTINCT
  groupName
FROM torrents
WHERE
  totalSnatched > 100 AND releaseType = "album"
sql:  SELECT
  groupName
FROM torrents
WHERE
  totalSnatched > 100
AND
  releaseType = "album" 1.0231418328883433
in execute query
cq:  STEP 1: 
Based on the previous clarification questions and incorrect queries, it is clear that the question is asking for albums that have been downloaded more than 100 times. The 'torrents' table contains information about albums, such as 'groupName' and 'totalSnatched'.

STEP 2:
- AmbQuestion: The question itself is clear.
- AmbTableColumn: There is no ambiguity in mapping the entities from the question to the tables and columns in the database schema.
- AmbOutput: It is unclear what fields should be contained in the output

sql:  SELECT
  groupName
FROM torrents
WHERE
  totalSnatched > 100
AND
  releaseType = "album" 1.0047777868985759
in execute query

------next question------

nl:  What is the average weight of players who were awarded third-base man in TSN all-star award? 7
gold:  SELECT
  AVG(T1.weight)
FROM player AS T1
JOIN player_award AS T2
  ON T1.player_id = T2.player_id
WHERE
  T2.award_id = "TSN All-Star" AND notes = "3B"
sql:  SELECT
  AVG(T3.weight)
FROM player_award AS T1
JOIN player AS T3
  ON T1.player_id = T3.player_id
WHERE
  T1.award_id = "TSN All-Star"
  AND T1.notes = "Third-base man" 1.0594390640318174
in execute query
cq:  STEP 1: 
Based on the previous clarification questions and incorrect queries, it is clear that the question is asking for the average weight of players who were awarded the third-base man in the TSN All-Star award. The player's weight information is stored in the 'weight' column of the 'player' table. The award information is stored in the 'player_award' table, 

sql:  SELECT
  AVG(T4.weight)
FROM player_award AS T1
JOIN player AS T4
  ON T1.player_id = T4.player_id
WHERE
  T1.award_id = "TSN All-Star"
  AND T1.notes = "Third-base man" 1.020096963829449
in execute query
cq:  STEP 1: 
- The column 'weight' in the 'player' table should be used to calculate the average weight of players.
- Players should be filtered based on both the 'award_id' and 'notes' columns in the 'player_award' table.
- The average weight of players should be calculated directly from the 'weight' column.

STEP 2:
- AmbQuestion: There is no ambiguity in the question itself.
- AmbTableColumn: The mapping of entities from the question to tables and columns in the database schema is clear.
- AmbOutput: The output table should contain a single column with the average weight of players.
- AmbValue: The predicate values for filtering based on 'award_id' and 'notes' are clear.

STEP 3:
mul_choice_cq = "How should the average weight of players be calculated for the TSN All-Star awa


feedback,  The portion of the gold query that answers the clarification question is: 
```sql
SELECT
  player_id
FROM player_award
GROUP BY
  player_id
HAVING
  COUNT(*) > 10
```
This part of the query is grouping the results by `player_id` and using the `HAVING` clause to filter those players who have been awarded more than ten times. It counts the number of awards per player and only includes those with more than ten awards in the count.

Now, considering the correctness of each multiple choice answer based on the gold query:

a) using the 'award_id' column with a predicate value of 'more than 10' - This choice is incorrect because the query does not filter by the 'award_id' column; it groups by 'player_id' and counts the awards.

b) using the 'year' column with a predicate value of 'more than 10' - This choice is incorrect because the query does not use the 'year' column for filtering or counting.

c) counting the number of awards per player and filtering those with more than ten aw

...


feedback,  The portion of the gold query that answers the clarification question is the SELECT and ORDER BY clauses, along with the LIMIT 1 statement. The SELECT clause specifies "growst" which indicates the state. The ORDER BY COUNT(*) DESC LIMIT 1 part of the query is used to order the states by the count of apples grown in descending order and then limit the result to only the top state. This means the query is designed to return only the state with the highest count of apples grown, without including the count itself.

Considering the correctness of each multiple choice answer based on the gold query:
- a) Yes, include the count of apples grown in each state - This choice is incorrect because the gold query does not include a command to select or display the count of apples grown in each state. It only selects the state (growst) and limits the result to the top one.
- b) No, only include the state with the highest count of apples grown - This choice is correct because the gold que


feedback,  The portion of the gold query that answers the clarification question is: ```WHERE commod = "AP"```. This line in the query filters the results based on the 'commod' column with the value 'AP', which is assumed to represent apples in this context.

Now, considering the correctness of each multiple choice answer based on the gold query:

a) Filter based on the 'commod' column with 'AP' value - This choice directly corresponds to the WHERE clause in the gold query, indicating it is the correct method for filtering the results to find the state with the most apples collected.

b) Filter based on the 'quantity' column with the highest value - The gold query does not mention or use the 'quantity' column for filtering or sorting. Therefore, this choice is incorrect.

c) Filter based on the 'state' column with the most occurrences - While the query does group by the 'distst' column (which could be interpreted as the 'state' column) and orders by the count of records to find the st

sql:  SELECT
  state
FROM sampledata15
WHERE commod = "AP"
GROUP BY state
ORDER BY COUNT(*) DESC
LIMIT 1 1.0012762000750417
in execute query

------next question------

nl:  Which varieties of apple typically have higher pesticide levels? 201
gold:  SELECT
  T1.variety
FROM resultsdata15 AS T2
JOIN sampledata15 AS T1
  ON T1.sample_pk = T2.sample_pk
WHERE
  T2.commod = "AP"
GROUP BY
  T1.variety
ORDER BY
  SUM(T2.concen) DESC
LIMIT 1
sql:  SELECT
  variety
FROM sampledata15
JOIN resultsdata15 ON sampledata15.sample_pk = resultsdata15.sample_pk
WHERE
  commod = "AP"
GROUP BY
  variety
ORDER BY
  AVG(concen) DESC 1.0420496719226815
in execute query
An error occurred during SQL execution: ambiguous column name: commod
After Fix Invalid:  sql = ```SELECT
  sampledata15.variety,
  AVG(resultsdata15.concen) AS avg_concen
FROM sampledata15
JOIN resultsdata15 ON sampledata15.sample_pk = resultsdata15.sample_pk
WHERE
  sampledata15.commod = 'AP'
GROUP BY
  sampledata15.variety
ORDER BY
  avg_co

sql:  SELECT
  sampledata15.variety
FROM sampledata15
JOIN resultsdata15 ON sampledata15.sample_pk = resultsdata15.sample_pk
WHERE
  sampledata15.commod = 'AP'
GROUP BY
  sampledata15.variety
ORDER BY
  SUM(resultsdata15.concen) DESC 1.0048240061949394
in execute query
cq:  STEP 1: 
- The pesticide levels should be compared by summing the concentration of pesticides.
- The output table should contain the variety name only.

STEP 2:
- AmbQuestion: The question itself is clear.
- AmbTableColumn: The mapping of entities from the question to tables and columns is clear (variety in sampledata15 table).
- AmbOutput: The output should only contain the variety name.
- AmbValue: The predicate value for filtering results is not specified.

STEP 3:
mul_choice_cq = "How should the pesticide levels be compared to determine which varieties of apples have higher levels? a) by taking the average concentration of pesticides, b) by summing the concentration of pesticides, c) by finding the maximum conce

in execute query
cq:  STEP 1: 
- The query should calculate the average pesticide residue for each state in the US where food is grown.
- The data is already filtered to include only the states in the US where food is grown.
- The average pesticide residue should be calculated based on the 'concen' column in the resultsdata15 table.

STEP 2:
- AmbQuestion: There is no ambiguity in the question itself.
- AmbColumn: The mapping of entities from the question to tables and columns is clear (resultsdata15 table, 'concen' column).
- AmbOutput: The output table should contain two fields: 'state' and 'avg_pesticide_residue'.
- AmbValue: There is no ambiguity in what predicate value should be used to filter the results.

STEP 3:
mul_choice_cq = "What should be included in the output table for each state? a) State and average pesticide residue, b) State only, c) Average pesticide residue only, d) Other (please specify)." 1.1288320952842736

feedback,  The clarification question is asking what sh

In [154]:
x = {}
for log in ['./fewshotlogs/kaggle_gpt35_baseline_4round_5shot.pkl',\
         './fewshotlogs/bird_gpt35_baseline_4round_5shot_from_devvect.pkl',\
         './fewshotlogs/kaggle_gpt35_interaction_4round_5shot.pkl',\
         './fewshotlogs/bird_gpt35_interaction_4round_5shot_from_devvect.pkl',\
         './fewshotlogs/kaggle_gpt35_interaction_4round_5shot_stop_early.pkl',\
         './fewshotlogs/bird_gpt35_interaction_4round_5shot_from_devvec_stop_early.pkl'\
         ]:
    with open(log, 'rb') as f:
        x = pickle.load(f)
    hist = collections.defaultdict(int)
    for k, v in x.items():
        hist[v['num_cq_asked']] += 1
    print(f, hist)
    print()

<_io.BufferedReader name='./fewshotlogs/kaggle_gpt35_baseline_4round_5shot.pkl'> defaultdict(<class 'int'>, {'Failed': 121, 0: 78, 4: 2, 1: 7})

<_io.BufferedReader name='./fewshotlogs/bird_gpt35_baseline_4round_5shot_from_devvect.pkl'> defaultdict(<class 'int'>, {'Failed': 108, 0: 62, 1: 6})

<_io.BufferedReader name='./fewshotlogs/kaggle_gpt35_interaction_4round_5shot.pkl'> defaultdict(<class 'int'>, {0: 82, 1: 39, 4: 3, 2: 16, 3: 7, 'Failed': 61})

<_io.BufferedReader name='./fewshotlogs/bird_gpt35_interaction_4round_5shot_from_devvect.pkl'> defaultdict(<class 'int'>, {'Failed': 88, 0: 64, 1: 10, 2: 9, 3: 2, 4: 3})

<_io.BufferedReader name='./fewshotlogs/kaggle_gpt35_interaction_4round_5shot_stop_early.pkl'> defaultdict(<class 'int'>, {'Failed': 86, 0: 78, 1: 26, 3: 5, 2: 13})

<_io.BufferedReader name='./fewshotlogs/bird_gpt35_interaction_4round_5shot_from_devvec_stop_early.pkl'> defaultdict(<class 'int'>, {'Failed': 98, 0: 64, 1: 8, 2: 4, 3: 1, 4: 1})



In [37]:
# kaggel_baseline_5shot_histories_gpt4_turbo = {}
# bird_baseline_5shot_histories_gpt4_turbo = {}
# bird_interaction_5shot_histories_gpt4_turbo = {}
# kaggle_interaction_5shot_histories_gpt4_turbo = {}
# kaggle_interaction_5shot_histories_stop_early_gpt4_turbo = {}
# bird_interaction_5shot_histories_stop_early_gpt4_turbo = {}

baselineFewShot(df, kaggel_baseline_5shot_histories_gpt4_turbo,\
                log_name='./fewshotlogs/kaggle_gpt4_turbo_baseline_4round_5shot.pkl', rounds=4,\
                num_of_tests=208, model_name='gpt4turbo', vectorstore=userstudy_vectorstore,\
                num_examples=5, data_source='kaggle')

baselineFewShot(bird_data_df, bird_baseline_5shot_histories_gpt4_turbo,\
                log_name='./fewshotlogs/bird_gpt4_turbo_baseline_4round_5shot_from_devvect.pkl', rounds=4,\
                num_of_tests=176, model_name='gpt4turbo', vectorstore=bird_dev_vectorstore,\
                num_examples=5, data_source='bird')


askClarificationQuestions(data_frame=df, history_log=kaggle_interaction_5shot_histories_gpt4_turbo,\
                  log_name='./fewshotlogs/kaggle_gpt4_turbo_interaction_4round_5shot.pkl', rounds=4,\
                  num_of_tests=208, model_name='gpt4turbo', vectorstore=userstudy_vectorstore, vectorstore_feedback=userstudy_vectorstore,\
                num_examples=5, data_source='kaggle', baseline_history=kaggel_baseline_5shot_histories_gpt4_turbo)


askClarificationQuestions(data_frame=bird_data_df, history_log=bird_interaction_5shot_histories_gpt4_turbo,\
                  log_name='./fewshotlogs/bird_gpt4_turbo_interaction_4round_5shot_from_devvect.pkl', rounds=4,\
                  num_of_tests=176, model_name='gpt4turbo', vectorstore=bird_dev_vectorstore, vectorstore_feedback=userstudy_vectorstore,\
                num_examples=5, data_source='bird', baseline_history=bird_baseline_5shot_histories_gpt4_turbo)

askCQsBreakNoAmb(data_frame=df, history_log=kaggle_interaction_5shot_histories_stop_early_gpt4_turbo,\
                  log_name='./fewshotlogs/kaggle_gpt4_turbo_interaction_4round_5shot_stop_early.pkl', rounds=4,\
                  num_of_tests=208, model_name='gpt4turbo', vectorstore=userstudy_vectorstore, vectorstore_feedback=userstudy_vectorstore,\
                num_examples=5, data_source='kaggle', baseline_history=kaggle_interaction_5shot_histories_gpt4_turbo)

askCQsBreakNoAmb(data_frame=bird_data_df, history_log=bird_interaction_5shot_histories_stop_early_gpt4_turbo,\
                  log_name='./fewshotlogs/bird_gpt4_turbo_interaction_4round_5shot_from_devvec_stop_early.pkl', rounds=4,\
                  num_of_tests=176, model_name='gpt4turbo', vectorstore=bird_dev_vectorstore, vectorstore_feedback=userstudy_vectorstore,\
                num_examples=5, data_source='bird', baseline_history=bird_interaction_5shot_histories_gpt4_turbo)

nl:  What defense position the player ID willite01 is? 9
gold:  SELECT DISTINCT
  notes
FROM player_award
WHERE
  player_id = "willite01"
sql:  
SELECT
  notes
FROM
  player_award
WHERE
  player_id = "willite01"
 1.040802294032983
in execute query
cq:  STEP 1: The question asks for the defense position of a specific player identified by the player ID "willite01". The incorrect query attempted to retrieve this information from the `notes` column of the `player_award` table, which suggests that the defense position might not be directly available in the queried column or table.

STEP 2:
- AmbQuestion: The question itself is clear in asking for the defense position of a specific player.
- AmbTableColumn: There is ambiguity in mapping the entities from the question to tables and columns in the database schema. It's not clear which table or column contains information about a player's defense position.
- AmbOutput: The question implies that the output should contain information about the de


feedback,  The gold query is selecting the "notes" column from the "player_award" table where the player_id matches "willite01". This suggests that the information sought about the player's defense position is expected to be found within the notes associated with awards the player has received.

Now, let's evaluate each multiple-choice option based on the gold query:

a) Any mention of a specific field position (e.g., 'Pitcher', 'Catcher', 'First Base') - This option is plausible because award notes could mention specific positions if the award is related to the player's performance in that position.

b) General terms like 'Defense' or 'Defensive' - This is also a plausible option because award notes could use general terms to describe a player's defensive capabilities or achievements without mentioning a specific position.

c) Awards specifically known for recognizing defensive skills - This option is relevant because the query is looking in the notes of awards, which could mention i


feedback,  The portion of the gold query that answers the clarification question is the `ORDER BY COUNT(*) DESC LIMIT 1` part. This part of the query is calculating the birth month of players who have been inducted into the hall of fame based on the total count of players inducted for each month. It does not calculate or consider the percentage of players inducted out of all players born in each month. Therefore, the query is directly counting the number of hall of fame inductees by birth month and then ordering these counts in descending order to find the top month. This approach aligns with option a) the total count of players inducted into the hall of fame born in each month.

Based on this reasoning, the correct answer to the clarification question, based solely on the gold query, is:

answer_to_cq = "a" 1.2581331465620194
sql:  ```sql
SELECT
  p.birth_month,
  COUNT(*) AS total_hof
FROM player AS p
JOIN hall_of_fame AS hof ON p.player_id = hof.player_id
WHERE hof.inducted = 'Y'
G

sql:  ```sql
SELECT
  notes AS position,
  AVG(CAST(weight AS INTEGER)) AS average_weight
FROM player_award
JOIN player
  ON player_award.player_id = player.player_id
GROUP BY
  notes
``` 1.0327328233811945
in execute query
cq:  STEP 1: 
From the previous clarification questions and user feedback, it is clear that the position of award-winning players should be inferred from the 'notes' or 'award_id' column in the 'player_award' table for calculating their average weight. The incorrect queries attempted to calculate the average weight of award-winning players by joining the 'player_award' and 'player' tables and grouping by the 'notes' column, which suggests that the 'notes' column was used to infer positions. However, these queries were marked incorrect, possibly due to incorrect interpretation of positions or how the join and grouping were handled.

STEP 2:
- AmbQuestion: The question itself seems clear - it asks for the average weight of award-winning players by their position.
- Am

sql:  ```sql
SELECT
  notes AS position,
  AVG(CAST(weight AS INTEGER)) AS average_weight
FROM player_award
JOIN player
  ON player_award.player_id = player.player_id
WHERE
  notes IS NOT NULL AND notes != ''
GROUP BY
  notes
``` 1.0323702278470663
in execute query
cq:  STEP 1: Summarize the information that is clear based on the answers to previous clarification questions and incorrect queries.

- It's clear that the position of award-winning players is to be inferred from the 'notes' column in the 'player_award' table.
- The entire content of the 'notes' column is considered indicative of the player's position.
- The calculation of average weight involves players identified in the 'player_award' table and their corresponding weight in the 'player' table.
- The incorrect queries attempted to group by the 'notes' column and calculate the average weight, but they were deemed incorrect, possibly due to not filtering out null or empty 'notes', not correctly handling the data types, or not

...


feedback,  The portion of the gold query that answers the clarification question is the `SELECT distst FROM sampledata15` part. This indicates that the column used to identify the state with the most apples collected is `distst`.

Given the options provided in the multiple choice clarification question:

a) Use the 'state' column to identify where the sample was collected,
b) Use the 'growst' column to identify where the apples were grown,
c) other (please specify).

Based on the gold query, neither 'state' nor 'growst' columns are mentioned or used. Instead, the column `distst` is used in the query to group the data and determine which state has the most apples collected. Therefore, both options a and b are incorrect based on the gold query provided.

The correct answer is based on the information directly from the gold query, which uses the `distst` column.

answer_to_cq = "c) other (please specify). Use the 'distst' column to identify the state." 1.2078002839153497
sql:  ```sql
SEL


feedback,  The gold query directly answers the natural language question by selecting commodities (`commod`) from a dataset (`resultsdata15`) where the concentration (`concen`) is greater than the limit of detection (`lod`). This query does not involve joining with another dataset or exclusively using another dataset to identify commodities. It simply checks if the concentration of pesticides in commodities is higher than their detection limits within the same dataset.

Now, let's evaluate each multiple-choice option based on the gold query:

a) This option suggests directly checking the concentration against the limit of detection in the results data without considering the sample data. This matches the approach taken in the gold query, where the condition `concen > lod` is used to filter the commodities directly within the `resultsdata15` dataset.

b) This option suggests joining the sample data with the results data to ensure accurate identification of commodities. The gold query d


feedback,  The gold query provided is: 
```sql
SELECT
  commod
FROM resultsdata15
WHERE
  concen > lod
```
This query selects commodities (`commod`) from a dataset (`resultsdata15`) where the concentration of pesticides (`concen`) is greater than the limit of detection (`lod`). 

Reasoning:
- The query does not specify any particular type of pesticide; it simply compares the concentration to the limit of detection for any and all pesticides recorded in the dataset.
- There is no filter or condition in the query that restricts the selection to specific types of pesticides.
- The query treats all pesticides equally by not differentiating between them in any way.

Correctness of each multiple choice answer based on the gold query:
- a) Focus on specific types of pesticides (please specify). This choice is incorrect because the query does not specify or focus on any particular type of pesticide.
- b) Consider all pesticides equally. This choice is correct because the query applies the sam


feedback,  Reasoning: The gold query selects the average concentration of pesticide residue (`AVG(T2.concen)`) for each state (`T1.growst`) from two tables joined on a common key (`T1.sample_pk = T2.sample_pk`). The query does not filter data based on the year, type of food, or any other specific conditions. It simply calculates the average pesticide residue for each state from the available data in the joined tables. Therefore, the query implies that all available data is used regardless of the year or type of food.

Considering the correctness of each multiple choice answer based on the gold query:
- a) All available data regardless of the year or type of food: This choice is correct because the query does not apply any filters on the year or type of food.
- b) Only data from a specific year or range of years: This choice is incorrect because the query does not specify any condition related to the year.
- c) Data for specific types of food (e.g., fruits, vegetables): This choice is 

sql:  ```sql
SELECT
  state,
  AVG(CASE WHEN concen > lod THEN concen ELSE NULL END) AS average_pesticide_residue
FROM
  sampledata15
JOIN
  resultsdata15 ON sampledata15.sample_pk = resultsdata15.sample_pk
WHERE
  concen > 0
GROUP BY
  state
``` 1.0376707467871442
in execute query
cq:  STEP 1: Summarize the information that is clear based on the answers to previous clarification questions and incorrect queries.

- The user wants to include all available data regardless of the year or type of food (from previous clarification question).
- The user wants the report to show only the name of the state and the average residue level (from previous clarification question).
- The user has a specific method in mind for handling measurements that are below the limit of detection (LOD), but this method was not specified (from previous clarification question).
- The incorrect queries attempt to calculate the average pesticide residue for each state by joining the `sampledata15` and `resultsdata15

In [38]:
x = {}
for log in ['./fewshotlogs/kaggle_gpt4_turbo_baseline_4round_5shot.pkl',\
         './fewshotlogs/bird_gpt4_turbo_baseline_4round_5shot_from_devvect.pkl',\
         './fewshotlogs/kaggle_gpt4_turbo_interaction_4round_5shot.pkl',\
         './fewshotlogs/bird_gpt4_turbo_interaction_4round_5shot_from_devvect.pkl',\
         './fewshotlogs/kaggle_gpt4_turbo_interaction_4round_5shot_stop_early.pkl',\
         './fewshotlogs/bird_gpt4_turbo_interaction_4round_5shot_from_devvec_stop_early.pkl'\
         ]:
    with open(log, 'rb') as f:
        x = pickle.load(f)
    hist = collections.defaultdict(int)
    for k, v in x.items():
        hist[v['num_cq_asked']] += 1
    print(f, hist)
    print()

<_io.BufferedReader name='./fewshotlogs/kaggle_gpt4_turbo_baseline_4round_5shot.pkl'> defaultdict(<class 'int'>, {1: 20, 3: 4, 0: 86, 'Failed': 86, 2: 10, 4: 2})

<_io.BufferedReader name='./fewshotlogs/bird_gpt4_turbo_baseline_4round_5shot_from_devvect.pkl'> defaultdict(<class 'int'>, {'Failed': 90, 0: 72, 2: 2, 1: 11, 3: 1})

<_io.BufferedReader name='./fewshotlogs/kaggle_gpt4_turbo_interaction_4round_5shot.pkl'> defaultdict(<class 'int'>, {1: 50, 0: 93, 3: 10, 2: 12, 4: 8, 'Failed': 35})

<_io.BufferedReader name='./fewshotlogs/bird_gpt4_turbo_interaction_4round_5shot_from_devvect.pkl'> defaultdict(<class 'int'>, {'Failed': 71, 0: 75, 2: 6, 1: 17, 4: 1, 3: 6})

<_io.BufferedReader name='./fewshotlogs/kaggle_gpt4_turbo_interaction_4round_5shot_stop_early.pkl'> defaultdict(<class 'int'>, {1: 48, 2: 12, 0: 86, 4: 3, 'Failed': 58, 3: 1})

<_io.BufferedReader name='./fewshotlogs/bird_gpt4_turbo_interaction_4round_5shot_from_devvec_stop_early.pkl'> defaultdict(<class 'int'>, {'Failed': 80